# Prerequisites

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project/validation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ece1786-project/validation


In [ ]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd
import torch 
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import evaluate
import numpy as np

In [ ]:
drinks_file = "drink_val.pkl"
cakes_file = "cake_val.pkl"
drink_df = pd.read_pickle(drinks_file)
cake_df = pd.read_pickle(cakes_file)
device = torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("../gpt2_combined")

#Functions

In [ ]:
def create_propmts(df, index, prefix):
    recipe = prefix + "RECIPE NAME\n" + df.loc[index, "Name"] + " \n\n" + prefix + "RECIPE INGREDIENTS\n"
    prompts = [recipe]
    for ingredient in df.loc[index, "Ingredients"]:
      recipe += ingredient + "\n"
    recipe += "\n" + prefix + "RECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[index, "Instructions"]:
      recipe += instruction + "\n"
    answers = [[recipe]]
    return answers, prompts

In [ ]:
def score(metric, df, index, model, prefix):
  answers, prompts = create_propmts(df, index, prefix)
  attempts = []
  encoded_prompt = tokenizer(prompts, return_tensors="pt").input_ids.to(device)
  ground_truth_length = (tokenizer(answers[0], return_tensors="pt").input_ids.to(device)).size(1)
  print(f"tokenized answer length: {ground_truth_length}")
  generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=ground_truth_length, temperature = 0.5, return_dict_in_generate=True, output_scores=True)
  decoded_sequence = tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0]
  attempts.append(decoded_sequence)
  
  eval = evaluate.load(metric)
  results = eval.compute(predictions=attempts, references=answers)
  return results, decoded_sequence

In [ ]:
def run(df, model, prefix, iterations=5, cardinality=10):
  predictions = []
  scores = np.zeros((cardinality, iterations))
  for i in range(cardinality):
    tmp = []
    for j in range(iterations):
      s, r = score("bleu", df, i, model, prefix)
      scores[i][j] = s["bleu"]
      tmp.append(r)
    predictions.append(tmp[np.argmax(scores[i])])
  return predictions, scores

#Drinks

In [ ]:
drink_answers, drink_scores = run(drink_df, model, prefix="DRINK ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 268


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 268


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

tokenized answer length: 130


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 130


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 130


In [ ]:
np.mean(drink_scores)

0.18935616254206317

In [ ]:
print(drink_answers[-1])

DRINK RECIPE NAME
Cider Delighter 

DRINK RECIPE INGREDIENTS
1/2 ounce Baileys Irish cream liqueur
1/2 ounce Kahlúa
1/2 ounce lemon juice, freshly squeezed
1/2 ounce honey syrup
Garnish: Baileys Irish cream liqueur, freshly squeezed

DRINK RECIPE INSTRUCTIONS
Add the Baileys, Irish cream liqueur, Kahlúa and honey syrup into a shaker with ice and shake until well-chilled.
Strain into a chilled cocktail


# Cake

In [ ]:
cake_answers, cake_scores = run(cake_df, model, prefix="CAKE ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 311


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 311


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

tokenized answer length: 454


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 454


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 454


In [ ]:
np.mean(cake_scores)

0.16061239084591997

In [ ]:
print(cake_answers[-1])

CAKE RECIPE NAME
Ricotta Pie 

CAKE RECIPE INGREDIENTS
1 ½ cups all-purpose flour
1 tablespoon baking powder
1 teaspoon baking soda
1 cup butter, softened
3  eggs
4  eggs
1 ½ cups milk
½ teaspoon salt
½ cup shortening
1 teaspoon vanilla extract
1 ½ cups white sugar

CAKE RECIPE INSTRUCTIONS
Preheat oven to 350 degrees F. Grease and flour a 9x13-inch baking dish.
Beat shortening and sugar together in a bowl using an electric mixer until light and fluffy. Beat in eggs, one at a time, mixing until fully incorporated; stir in vanilla. Combine flour, baking powder, baking soda, and salt; stir into butter mixture alternately with milk. Spread batter into the prepared baking dish.
Bake in the preheated oven until a toothpick inserted into the center comes out clean, about 20 minutes. Cool on a wire rack for 5 minutes. Run a table knife around the edges to loosen, then invert carefully onto a serving plate or cooling rack. Let cool completely.



#Writing to file

In [ ]:
combined_answers = drink_answers + cake_answers
with open('combined_val_results.txt', 'w') as fp:
    for item in combined_answers:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
